In [3]:
from scipy.signal import iirnotch, butter, filtfilt
import numpy as np

class BandFilter:
    def __init__(self, order=4, fc_bp=[20, 480], freq=1000):
        nyq = 0.5 * freq
        low = fc_bp[0] / nyq
        high = fc_bp[1] / nyq
        self.b, self.a = butter(order, (low, high), btype = 'bandpass', output='ba')
    
    def filtfilt(self, x):
        filtered_x = filtfilt(self.b, self.a, x)
        return filtered_x
    
class LowPassFilter:
    def __init__(self, order=2, fc_bp=4, freq=1000):
        nyq = 0.5 * freq
        low = fc_bp / nyq
        self.b, self.a = butter(order, low, btype = 'lowpass', output='ba')
    
    def filtfilt(self, x):
        filtered_x = filtfilt(self.b, self.a, x)
        return filtered_x

class HighPassFilter:
    def __init__(self, order=2, fc_bp=30, freq=1000):
        nyq = 0.5 * freq
        high = fc_bp / nyq
        self.b, self.a = butter(order, high, btype = 'highpass', output='ba')
    
    def filtfilt(self, x):
        filtered_x = filtfilt(self.b, self.a, x)
        return filtered_x

class NotchFilter:
    def __init__(self, f0=60, freq=1000):
        f0 = f0  # Frequency to be removed from signal (Hz)
        Q = 30.0  # Quality factor
        self.b, self.a = iirnotch(f0, Q, freq)
    
    def filtfilt(self, x):
        filtered_x = filtfilt(self.b, self.a, x)
        return filtered_x
    
class EMGProcessor:
    def __init__(self, sampling_rate=1000, highpass_order=4, highpass_fc=30,
                 lowpass_order=4, lowpass_fc=4, notch_f0=60):
        self.h_filter = HighPassFilter(order=highpass_order, fc_bp=highpass_fc, freq=sampling_rate)
        self.l_filter = LowPassFilter(order=lowpass_order, fc_bp=lowpass_fc, freq=sampling_rate)
        self.n_filter = NotchFilter(f0=notch_f0, freq=sampling_rate)

    def process(self, data):
        # Apply high-pass filter
        filted_emg = self.h_filter.filtfilt(data)
        # Apply notch filter
        filted_emg = self.n_filter.filtfilt(filted_emg)
        # Rectify the EMG signal
        rect_emg = np.abs(filted_emg)
        # Get the envelope using low-pass filter
        envelope = self.l_filter.filtfilt(rect_emg)
        return envelope
    
    def normalize_data(self, data):
        min_val = np.min(data)
        max_val = np.max(data)
        if max_val == min_val:
            return data
        return (data - min_val) / (max_val - min_val)

In [2]:
def modify_trc_file(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        header = []

        for _ in range(5):
            line = infile.readline()
            header.append(line)
            outfile.write(line)
        
        for line in infile:
            columns = line.strip().split('\t')
            
            for i in range(2, len(columns)): 
                try:
                    value = float(columns[i])
                    if (i - 2) % 3 == 0 or (i - 2) % 3 == 2:
                        columns[i] = str(value * -1)
                except ValueError:
                    pass
            
            outfile.write('\t'.join(columns) + '\n')

input_trc = 'data/Empty_project_filt_0-30.trc'
output_trc = 'output/walking_modified.trc'

modify_trc_file(input_trc, output_trc)

In [4]:
import pandas as pd
from scipy.signal import resample

csv_file = 'data/cutted_EMG_data.csv'
df = pd.read_csv(csv_file)

csv_columns = ['LGL', 'RGL', 'LTA', 'RTA', 'LBF', 'RBF', 'LRF', 'RRF']
sto_columns = ['gastroc_l', 'gastroc_r', 'tib_ant_l', 'tib_ant_r', 'hamstrings_l', 'hamstrings_r', 'rect_fem_l', 'rect_fem_r']

original_rate = 1000
target_rate = 30
num_samples = int(len(df) * target_rate / original_rate)

processor = EMGProcessor()

resampled_muscle_data = {}
for muscle in csv_columns:
    processed_signal = processor.process(df[muscle].values)
    processed_signal = processor.normalize_data(processed_signal)
    resampled_muscle_data[muscle] = resample(processed_signal, num_samples)

resampled_time = [i * (1/30) for i in range(num_samples)]

sto_file = 'output/resampled_emg_custom_time.sto'

header = """version=1
nRows={}
nColumns={}
inDegrees=no

Units are S.I. units (second, meters, Newtons, ...)
endheader
time\t{}\n""".format(num_samples, len(sto_columns) + 1, '\t'.join(sto_columns))

with open(sto_file, 'w') as f:
    f.write(header)
    
    for i in range(num_samples):
        row = [f"{resampled_time[i]:.6f}"]
        row += [f"{resampled_muscle_data[muscle][i]:.6f}" for muscle in csv_columns]
        f.write('\t'.join(row) + '\n')

print(f"EMG数据已保存为: {sto_file}")

EMG数据已保存为: output/resampled_emg_custom_time.sto


In [9]:
import pandas as pd
def extract_and_sort_sto_data(file_path, start_time, end_time, output_file):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    header_lines = lines[:7]
    df = pd.read_csv(file_path, delim_whitespace=True, comment='%', skiprows=7)
    if 'time' not in df.columns:
        raise ValueError("The .sto file does not contain a 'time' column.")
    df_filtered = df[(df['time'] >= start_time) & (df['time'] <= end_time)]
    df_sorted = df_filtered.sort_values(by='time').reset_index(drop=True)
    original_time_diff = df_sorted['time'].diff().dropna().mean()
    df_sorted['time'] = (1/30) + df_sorted.index * original_time_diff
    df_sorted['time'] = df_sorted['time'].round(6)  # 將時間列的數據保留小數點後6位
    with open(output_file, 'w') as f_out:
        f_out.writelines(header_lines)
        df_sorted.to_csv(f_out, index=False, sep='\t', lineterminator='\n')

file_path = 'data/resampled_emg_custom_time.sto'
start_time = 12.066667
end_time = 16.766667
output_file = 'data/emg_cut.sto'

extract_and_sort_sto_data(file_path, start_time, end_time, output_file)


C:\Users\sean9\AppData\Local\Temp\ipykernel_6544\3819528721.py:6: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(file_path, delim_whitespace=True, comment='%', skiprows=7)
